# Graph Homework Notebook

By: Nicholas Soucy

This notebook allows the user to impliment graphs in three different representation types, Adjacency List, Edge List, and Adjacency Matrix. The user is able to create weighted, unweighted and directed or undirected graphs using any of the three graph representation types.

All graph representation classes are subclasses of 'Graph'. Each graph representation supports a number of graph related functions. These functions are: adding and deleting a vertex or edge, reporting all vertices, reporting all edges, return a list description of the graph, finding an edge or vertex, return a list of incident edges and adjacent vertices, return the cost of an edge for a weighted graph, return the path cost of a list of vertices, and adjacent to and incident to functions that are boolean functions.

In addition to those graph related functions, a depth first traversal method is implimented to have any graph representation be passed and return a spanning tree around the given vertex. 

## Given Graph Class Code

In [1]:
class Graph ():

    def __init__(self,graph_desc=None,properties=None):

        self.properties = properties

        self.vertices = []      #List of vertex objects to preserve order in graph             

        if graph_desc:
            if properties == None:
                self.properties = [graph_desc[0]]
            else:
                self.properties.append(graph_desc[0])
    
    def find_vertex(self,name):
        for v in range(0,len(self.vertices)):
            if self.vertices[v].name == name:
                return v
        return None

    def all_vertices(self,desc):
        all_vert = []
        if desc == ['name']:
            for v in range(0,len(self.vertices)):
                all_vert.append(self.vertices[v].name)
        elif desc == ['vert']:
            for v in range(0,len(self.vertices)):
                all_vert.append(Vertex(self,self.vertices[v].name))

        return all_vert

    def incident_to(self,vertex,edge):
        if isinstance(edge, Edge):
            if self.find_edge(edge.v1,edge.v2) is not None:
                if vertex == edge.v1 or vertex == edge.v2:
                    return True
                else:
                    return False
            else:
                return False
        else:
            if self.find_edge(edge[0],edge[1]) is not None:
                if vertex == edge[0] or vertex == edge[1]:
                    return True
                else:
                    return False
            else:
                return False

    def to_list(self):
        if 'digraph' in self.properties:
            output = ['digraph']
        else:
            output = ['graph']

        for vname in self.vertices:
            output.append([vname.name])

        ListEdge = self.all_edges(['name'])
        for e in ListEdge:
            output.append(e)

        return output

    def to_gv(self):
        desc = self.to_list()

        print(desc[0], " {")

        if desc[0] == 'digraph':
            sep = " -> "
        else:
            sep = " -- "
            
        for thing in desc[1:]:
            if len(thing) == 1:  # vertex
                print('   ', thing[0])
            elif len(thing) > 2: # edge, weight exists
                print('   ', thing[0], sep, thing[1], ' ', '[weight="', thing[2],
                      '"]')
            else: #no weight
                print('   ', thing[0],sep,thing[1])
        print('}')
    
    def dft(self,vertex):
        global time
        global span
        
        #create the spanning tree based on the type of graph and graph representation
        if isinstance(self,AdjList):
            if 'digraph' in self.properties:
                span = AdjList(['digraph'])
            else:
                span = AdjList(['graph'])
        elif isinstance(self,EdgeList):
            if 'digraph' in self.properties:
                span = EdgeList(['digraph'])
            else:
                span = EdgeList(['graph'])
        elif isinstance(self,AdjMatrix):
            if 'digraph' in self.properties:
                span = AdjMatrix(['digraph'])
            else:
                span = AdjMatrix(['graph'])


        #Reset all dft vertex variables
        for v in self.vertices:
            v.color = 'black'
            v.discovery = 0
            v.finish = 0

        time = 1
        self._dft(vertex)
        #all vertices are blue, return spanning tree
        return span

    def _dft(self,vertex):
        global time
        global span
        #first time visiting vertex, paint red, then add to spanning tree
        self.vertices[self.find_vertex(vertex)].color = 'red'
        span.add_vertex(vertex)
        self.vertices[self.find_vertex(vertex)].discovery = time
        time += 1
        #search through the neighbors of the vertex and recursively call
        for v in self.adjacent_vertices(vertex): 
            if self.vertices[self.find_vertex(v)].color == 'black':
                if self.edge_cost(vertex,v) == -1:
                    span.add_edge(vertex,v)
                else:
                    span.add_edge(vertex,v,self.edge_cost(vertex,v))
                self._dft(v)
        #all neighbors of current vertex have been vistied, paint blue
        self.vertices[self.find_vertex(vertex)].color = 'blue'
        self.vertices[self.find_vertex(vertex)].finish = time
        time += 1

Here are the Vertex and Edge classes:

In [2]:
class Vertex ():
    def __init__(self,graph,name,color=None):
        #For graph
        self.graph = graph
        self.name = name

        #For dft
        self.discovery = 0
        self.finish = 0
        self.color = 'black'

class Edge ():
    def __init__(self,graph,v1,v2,weight=None):
        self.graph = graph
        self.v1 = v1
        self.v2 = v2
        self.weight = weight

## Homework Classes

### Adjacency List Class

The Adjacency List Class impliments the graph via a list of list. The named vertices are stored in a vertex list in the form of a Vertex object. The Adjacency list class uses only numbers to repersent vertices in the graph, using the index of the named vertex in the vertex list as the number. This allows us to perform numeric operations easily, while having the user being able to input named vertices.

The Adjacency List will be a list where each index is the base vertices. In each element, there will be a list of all vertices that are connected to the given vertex, along with the weight of the edge.

In [3]:
class AdjList(Graph):
    def __init__(self,graph_desc=None,properties=None):
        super().__init__(graph_desc,properties)
        self.adj_list = []

    def add_vertex(self,name):
        for v in self.vertices:
            if v.name == name:
                return None
            
        self.vertices.append(Vertex(self,name))
        self.adj_list.append([])

    def delete_vertex(self,name):
        if self.find_vertex(name) is not None:
            #remove all edges that connect to that vertex
            for i in range (0,len(self.adj_list)):
                for j in self.adj_list[i].copy():
                    #if the element contain the vertex you want to delete, remove it
                    if self.find_vertex(name) == j[0]:
                        self.adj_list[i].remove(j)
                    #if it doesnt contain the vertex you want to delete, shift it.
                    elif j[0] > self.find_vertex(name):
                        j[0] = j[0] -1

            #remove vertex from adj list
            del self.adj_list[self.find_vertex(name)]

            #remove vertex from vert list
            del self.vertices[self.find_vertex(name)]
        else:
            print("Vertex "+name+" does not exist!")

    def add_edge(self,v1,v2,weight=None):
        #if vertex v1 doesnt exist, add it
        if self.find_vertex(v1) == None:
            self.add_vertex(v1)
        #if vertex v2 doesnt exist, add it
        if self.find_vertex(v2) == None:
            self.add_vertex(v2)
        #if edge doesn't already exist, add it
        if self.find_edge(v1,v2) == None:
            self.adj_list[self.find_vertex(v1)].append([self.find_vertex(v2),weight])
            if 'digraph' not in self.properties:
                self.adj_list[self.find_vertex(v2)].append([self.find_vertex(v1),weight])

    def delete_edge(self,v1,v2):
        #Search in the list of the given source vertex
        if (self.find_vertex(v1) is not None) and (self.find_vertex(v2) is not None) and (self.find_edge(v1,v2) is not None):
            for i in self.adj_list[self.find_vertex(v1)].copy():
                #if the dest vertex is in the source list, delete it
                if self.find_vertex(v2) == i[0]:
                    self.adj_list[self.find_vertex(v1)].remove(i)
                    break
                    #if it is an undirected graph
            if 'digraph' not in self.properties:
                #search through the list of the dest vertex
                for j in self.adj_list[self.find_vertex(v2)].copy():
                    #if the source vertex is in the dest vertex list, delete it
                    if self.find_vertex(v1) == j[0]:
                        self.adj_list[self.find_vertex(v2)].remove(j)
                        break
        else:
            print("Edge from "+v1+" to "+v2+" does not exist!")

    def all_edges(self,spec):
        list_edges = []
        if spec == ['name']: 
            #go through each element in list
            for i in range (0,len(self.adj_list)):
                #go through each element in the list of list
                for j in range (0,len(self.adj_list[i])):
                    #append edge info to edge list
                    list_edges.append([self.vertices[i].name,self.vertices[self.adj_list[i][j][0]].name,self.adj_list[i][j][1]])
        elif spec == ['edge']:
            #go through each element in list
            for i in range (0,len(self.adj_list)):
                #go through each element in the list of list
                for j in range (0,len(self.adj_list[i])):
                    #append edge info to edge list
                    list_edges.append(Edge(self.vertices[i].name,self.vertices[self.adj_list[i][j][0]].name,self.adj_list[i][j][1]))

        return list_edges

    def find_edge(self,v1,v2,weight=None):
        if self.find_vertex(v1) != None and self.find_vertex(v2) != None:
            #Search in the list of the given source vertex
            for i in range (0,len(self.adj_list[self.find_vertex(v1)])):
                #if the dest vertex is in the source list, delete it
                if self.find_vertex(v2) in self.adj_list[self.find_vertex(v1)][i]:
                    return True
            return None
        else:
            return None

    def incident_edges(self,v):
        if self.find_vertex(v) is not None:
            incident = []
            for e in range (0,len(self.adj_list[self.find_vertex(v)])):
                incident.append([v,self.vertices[self.adj_list[self.find_vertex(v)][e][0]].name])
            return incident
        else:
            print("Vertex "+v+" does not exist!")

    def adjacent_vertices(self,v):
        if self.find_vertex(v) is not None:
            adj_vert = []
            for e in range (0,len(self.adj_list[self.find_vertex(v)])):
                adj_vert.append(self.vertices[self.adj_list[self.find_vertex(v)][e][0]].name)
            return adj_vert
        else:
            print("Vertex "+v+" does not exist!")

    def edge_cost(self,v1,v2):
        if self.find_edge(v1,v2) is not None:
            for i in range (0,len(self.adj_list[self.find_vertex(v1)])):
                if self.find_vertex(v2) in self.adj_list[self.find_vertex(v1)][i]:
                    return self.adj_list[self.find_vertex(v1)][i][1]
        else:
            print("Edge from "+v1+" to "+v2+" does not exist!")

    def path_cost(self,vert_list):
        distance = 0
        for i in range (0,len(vert_list) - 1):
            distance += self.edge_cost(vert_list[i],vert_list[i+1])
        return distance

    def adjacent_to(self,v1,v2):
        if (self.find_vertex(v1) is not None) and (self.find_vertex(v2) is not None):
            for e in range (0,len(self.adj_list[self.find_vertex(v1)])):
                if self.find_vertex(v2) == self.adj_list[self.find_vertex(v1)][e][0]:
                    return True
            return False
        else:
            return False

### Edge List Class

Similarly to the Adjacency list class, the named vertices are stored in a vertex list in the form of a Vertex object. The Edge list class uses only numbers to repersent vertices in the graph, using the index of the named vertex in the vertex list as the number.

The Edge List class is implimented via a list of Edge objects, where each element contains the source and destination vertex names.

In [4]:
class EdgeList(Graph):

    def __init__(self,graph_desc=None,properties=None):
       super().__init__(graph_desc,properties)
       self.edges = []

    def add_vertex(self,name):
        for v in self.vertices:
            if v.name == name:
                return None
        self.vertices.append(Vertex(self,name))

    def delete_vertex(self,v_name):
        if self.find_vertex(v_name) is not None:
            for e in self.edges.copy():
                #delete if edge contains vertex
                if e.v1 == self.find_vertex(v_name) or e.v2 == self.find_vertex(v_name):
                    self.edges.remove(e)
                #shift all index values to perseve order, if you don't delete it
                else:
                    if e.v1 > self.find_vertex(v_name):
                        e.v1 = e.v1 - 1
                    if e.v2 > self.find_vertex(v_name):
                        e.v2 = e.v2 - 1

            #delete vertex from list
            del self.vertices[self.find_vertex(v_name)]
        else:
            print("Vertex "+v_name+" does not exist!")
        
    def add_edge(self,v1,v2,weight=None):
        source = self.find_vertex(v1)
        dest = self.find_vertex(v2)
        #if vertex v1 doesnt exist, add it
        if source == None:
            self.add_vertex(v1)
            source = self.find_vertex(v1)
        #if vertex v2 doesnt exist, add it
        if dest == None:
            self.add_vertex(v2)
            dest = self.find_vertex(v2)
        #if edge doesn't already exist, add it
        if self.find_edge(v1,v2) == None:
            self.edges.append(Edge(self,source,dest,weight))
            if 'digraph' not in self.properties:
                self.edges.append(Edge(self,dest,source,weight))

    def delete_edge(self,v1,v2):
        if (self.find_vertex(v1) is not None) and (self.find_vertex(v2) is not None) and (self.find_edge(v1,v2) is not None):
            e = self.find_edge(v1,v2)
            if e is not None:
                del self.edges[e]
            if 'digraph' not in self.properties:
                e = self.find_edge(v2,v1)
                if e is not None:
                    del self.edges[e]
        else:
            print("Edge from "+v1+" to "+v2+" does not exist!")

    def all_edges(self,spec):
        all_edges = []
        if spec == ['name']:
            for e in self.edges:
                all_edges.append([self.vertices[e.v1].name,self.vertices[e.v2].name,e.weight])
        elif spec == ['edge']:
            for e in self.edges:
                all_edges.append(Edge(self.vertices[e.v1].name,self.vertices[e.v2].name,e.weight))
        return all_edges

    def find_edge(self,v1,v2):
        #iterate through the entire list
        for e in range(0,len(self.edges)):
            #if the edge matches the v1,v2 pair, return index
            if self.edges[e].v1 == self.find_vertex(v1) and self.edges[e].v2 == self.find_vertex(v2):
              return e
        return None

    def incident_edges(self,v):
        if self.find_vertex(v) is not None:
            incident = []
            for e in range (0,len(self.edges)):
                if (self.edges[e].v1 == self.find_vertex(v)) or (self.edges[e].v2 == self.find_vertex(v)):
                    incident.append([self.vertices[self.edges[e].v1].name,self.vertices[self.edges[e].v2].name])
            return incident
        else:
            print("Vertex "+v+" does not exist!")

    def adjacent_vertices(self,v):
        if self.find_vertex(v) is not None:
            adj_vert = []
            for e in range (0, len(self.edges)):
                if (self.edges[e].v1 == self.find_vertex(v)) and (self.find_vertex(v) not in adj_vert):
                    adj_vert.append(self.vertices[self.edges[e].v2].name)
            return adj_vert
        else:
            print("Vertex "+v+" does not exist!")

    def edge_cost(self,v1,v2):
        if self.find_edge(v1,v2) is not None:
            return self.edges[self.find_edge(v1,v2)].weight
        else:
            print("Edge from "+v1+" to "+v2+" does not exist!")

    def path_cost(self,vert_list):
        distance = 0
        for i in range (0,len(vert_list) - 1):
            distance += self.edge_cost(vert_list[i],vert_list[i+1])
        return distance

    def adjacent_to(self,v1,v2):
        if (self.find_vertex(v1) is not None) and (self.find_vertex(v2) is not None):
            if self.find_edge(v1,v2) is not None:
                return True
            else:
                return False
        else:
            return False

### Adjacency Matrix Class

Similarly to the list classes above, the named vertices are stored in a vertex list in the form of a Vertex object. The Adjacency Matrix class uses only numbers to repersent vertices in the graph, using the index of the named vertex in the vertex list as the number.

The Adjacency Matrix Class uses a square 2D array where the length is the number of vertices. Each element in the matrix will be zero if there is no edge between the two vertices, -1 if there is an unweighted edge between the two vertices, or the weight if there is a weighted edge between the two vertices. For example if there is an edge between v1 and v2, Matrix[v1][v2] would equal -1, or the weight.

In [5]:
class AdjMatrix(Graph):
    def __init__(self,graph_desc=None,properties=None):
        super().__init__(graph_desc,properties)
        self.matrix = []                       #2d array reperesenting adjacency matrix

    def add_vertex(self,name):
        #if the name isn't already in the graph
        for v in self.vertices:
            if v.name == name:
                return None

        #append new vertex
        self.vertices.append(Vertex(self,name))
        
        #increase size of 2d array by one
        temp = [x[:] for x in [[0] * len(self.vertices)] * len(self.vertices)]
        for r in range (0,len(self.vertices)-1):
            for c in range (0,len(self.vertices)-1):
                temp[r][c] = self.matrix[r][c]
        self.matrix = temp           

    def delete_vertex(self,name):
        #if the vertex actually exist
        if self.find_vertex(name) is not None:
            #decrease size of 2d array by one
            for r in range (0,len(self.vertices)):
                del self.matrix[r][self.find_vertex(name)]
            
            del self.matrix[self.find_vertex(name)]

            #remove vertex from vertex list, then shift all memebers to the left
            del self.vertices[self.find_vertex(name)]
            #self.vertices.remove(name)

    def add_edge(self,v1,v2,weight=None):
        if weight is None:
            weight = -1

        #update v1 and v2 to be the indexes for the vertices
        v1_i = self.find_vertex(v1)
        v2_i = self.find_vertex(v2)

        #check to see if vertices actually exist, if not, add them
        if v1_i is None:
            self.add_vertex(v1)
            v1_i = self.find_vertex(v1)
        if v2_i is None:
            self.add_vertex(v2)
            v2_i = self.find_vertex(v2)
        
        #if edge doens't already exist, add to matrix
        if self.find_edge(v1,v2) == None:
            self.matrix[v1_i][v2_i] = weight
            if 'digraph' not in self.properties:
                self.matrix[v2_i][v1_i] = weight

    def delete_edge(self,v1,v2):
        
        #update v1 and v2 to be the indexes for the vertices
        v1_i = self.find_vertex(v1)
        v2_i = self.find_vertex(v2)

        if v1_i != None and v2_i != None:
            #delete edge from matrix
            self.matrix[v1_i][v2_i] = 0
            if 'digraph' not in self.properties:
                self.matrix[v2_i][v1_i] = 0

    def all_edges(self,spec):
        all_edges=[]
        if spec == ['name']:
            for r in range(len(self.vertices)):
                for c in range(len(self.vertices)):
                    if self.matrix[r][c] > 0:
                        all_edges.append([self.vertices[r].name,self.vertices[c].name,self.matrix[r][c]]) 
                    elif self.matrix[r][c] == -1:
                        all_edges.append([self.vertices[r].name,self.vertices[c].name,None])
        elif spec == ['edge']:
            for r in range(len(self.vertices)):
                for c in range(len(self.vertices)):
                    if self.matrix[r][c] > 0:
                        all_edges.append(Edge(self.vertices[r].name,self.vertices[c].name,self.matrix[r][c])) 
                    elif self.matrix[r][c] == -1:
                        all_edges.append(Edge(self.vertices[r].name,self.vertices[c].name,None))
        return all_edges

    def find_edge(self,v1,v2,weight=None):
        if self.find_vertex(v1) != None and self.find_vertex(v2) != None:
            if (self.matrix[self.find_vertex(v1)][self.find_vertex(v2)] > 0) or (self.matrix[self.find_vertex(v1)][self.find_vertex(v2)] == -1):
                return True
            else:
                return None
        else:
            return None

    def incident_edges(self,vertex):
        if self.find_vertex(vertex) is not None:
            edge_list = []
            for c in range (0, len(self.vertices)):
                if self.matrix[self.find_vertex(vertex)][c] > 0:
                    edge_list.append([vertex,self.vertices[c].name])
            return edge_list
        else:
            print("Vertex "+vertex+" does not exist!")

    def adjacent_vertices(self,v):
        if self.find_vertex(v) is not None:
            adj_vert = []
            for c in range (0, len(self.vertices)):
                if (self.matrix[self.find_vertex(v)][c] > 0) or (self.matrix[self.find_vertex(v)][c] == -1):
                    adj_vert.append(self.vertices[c].name)
            return adj_vert
        else:
            print("Vertex "+v+" does not exist!")

    def edge_cost(self,v1,v2):
        #update v1 and v2 to be the indexes for the vertices
        v1_i = self.find_vertex(v1)
        v2_i = self.find_vertex(v2)
        if v1_i != None and v2_i != None:
            return self.matrix[v1_i][v2_i]

    def path_cost(self,vert_list):
        distance = 0
        for i in range (0,len(vert_list) - 1):
            distance += self.edge_cost(vert_list[i],vert_list[i+1])
            #if graph is not weighted
            if distance == -1:
                return None
        return distance

    def adjacent_to(self,v1,v2):
        if (self.find_vertex(v1) is not None) and (self.find_vertex(v2) is not None):
            if self.matrix[self.find_vertex(v1)][self.find_vertex(v2)] > 0:
                return True
            else:
                return False
        else:
            return False

## Driver Code

This section will teach us how to create and use these different classes.

Lets start by creating a few graphs...

In [6]:
a = AdjList(['graph'])

This will create a undirected graph, it can be weighted or unweighted based on how we add edges, we will look at that later.

Lets make directed graph now.

In [7]:
b = AdjList(['digraph'])

This will create a directed graph, it can be weighted or unweighted based on how we add edges, like before.

You can make the Edge List and Adjacency Matrix graphs the same way you can the Adjacency List as shown above, like this:

In [8]:
c = EdgeList(['graph'])
d = EdgeList(['digraph'])
e = AdjMatrix(['graph'])
f = AdjMatrix(['digraph'])

Everything that we do to each graph will be the exact same, so lets just use our original Adjacency List example.

Now lets add some vertices to our Adjacency List graph. Vertices can be named anything we want.

In [9]:
a.add_vertex('first')
a.add_vertex('second')
a.add_vertex('third')

Lets also add these same vertices to our directed graph so we can see the difference between the two later.

In [10]:
b.add_vertex('first')
b.add_vertex('second')
b.add_vertex('third')

This is a pretty boring graph, so lets add some edges. For example's sake, lets have our undirect graph an unweighted one with our direct graph being a weighted one.

In [11]:
a.add_edge('first','second')
a.add_edge('second','third')
a.add_edge('third','first')

b.add_edge('first','second',3)
b.add_edge('second','third',3)
b.add_edge('third','first',1)

Adding edges are pretty straight forward (pun intended), we can see weights are just a third optional argument.

We now have a nice triangle graph. A nice thing to do would be to print out our results so we can see what the graph looks like. Lets start by printing our vertices and edges.

In [12]:
print('***Undirected Graph***')
print(a.all_vertices(['name']))
print('\n')
print(a.all_edges(['name']))
print('\n')
print('***Directed Graph***')
print(b.all_vertices(['name']))
print('\n')
print(b.all_edges(['name']))

***Undirected Graph***
['first', 'second', 'third']


[['first', 'second', None], ['first', 'third', None], ['second', 'first', None], ['second', 'third', None], ['third', 'second', None], ['third', 'first', None]]


***Directed Graph***
['first', 'second', 'third']


[['first', 'second', 3], ['second', 'third', 3], ['third', 'first', 1]]


We can see that our unweighted graph just prints 'None' if there is no edge weight. We can also see that when we added an edge to our undirected graph, it also added its inverse. This is very handy!

NOTE: for the all_vertices and all_edges functions, for arguments they take ['vert'] and ['edge'] respectively to return a list of Vertex or Edge objects. They also both take ['name'] to return a list of the vertex names rather than Vertex or Edge objects.

We can also print a list version of the graph as shown below:

In [13]:
print('***Undirected Graph***')
print(a.to_list())
print('\n')
print('***Directed Graph***')
print(b.to_list())

***Undirected Graph***
['graph', ['first'], ['second'], ['third'], ['first', 'second', None], ['first', 'third', None], ['second', 'first', None], ['second', 'third', None], ['third', 'second', None], ['third', 'first', None]]


***Directed Graph***
['digraph', ['first'], ['second'], ['third'], ['first', 'second', 3], ['second', 'third', 3], ['third', 'first', 1]]


This can be useful, but even more usefull is the print function to_gv(), lets print with that instead.

In [14]:
print('***Undirected Graph***')
print(a.to_gv())
print('\n')
print('***Directed Graph***')
print(b.to_gv())

***Undirected Graph***
graph  {
    first
    second
    third
    first  --  second   [weight=" None "]
    first  --  third   [weight=" None "]
    second  --  first   [weight=" None "]
    second  --  third   [weight=" None "]
    third  --  second   [weight=" None "]
    third  --  first   [weight=" None "]
}
None


***Directed Graph***
digraph  {
    first
    second
    third
    first  ->  second   [weight=" 3 "]
    second  ->  third   [weight=" 3 "]
    third  ->  first   [weight=" 1 "]
}
None


Much better! This will be used a lot during your work!

For the find_edge and find_vertex functions.

The find_vertex function takes a vertex as the argument and returns the index of the vertex in the vertex list or None if the vertex does not exist.

In [15]:
print(a.find_vertex('first'))
print(a.find_vertex('foo'))

0
None


Find_edge is very similar, it will return None if the edge is not found. For the AdjList and AdjMatrix, it will return True if it does exist. If you are using the EdgeList, it will return the index of the Edge in the edge list.

In [16]:
print(a.find_edge('first','second'))
print(a.find_edge('foo','bar'))

True
None


Lets now get to our more heavy graph theory functions like incident_edges, adjacent_vertices, incident_to and adjacent_to. 

Lets use our undirect graph for these examples, starting with incident_edges and incident_to.

Incident_Edges will return a list of all edges incident on a vertex, so lets see what vertices are incident on the vertex 'first'.

In [17]:
print(a.incident_edges('first'))
print('\n'+'Is vertex \'second\' incident on \'first\'?\n')
print(a.incident_to('first',['first','second']))

[['first', 'second'], ['first', 'third']]

Is vertex 'second' incident on 'first'?

True


Now for our adjacent_vertices and adjacent_to.

In [18]:
print(a.adjacent_vertices('first'))
print('\n'+'Is vertex \'second\' adjacent to \'first\'?\n')
print(a.adjacent_to('first','second'))

['second', 'third']

Is vertex 'second' adjacent to 'first'?

True


The next two functions are edge_cost, which returns the weight of a given edge, and path cost, which returns the weighted cost of a given path. We will have to use our directed weighted graph b.

In [19]:
print('Edge cost of \'first\' -> \'second\'')
print(b.edge_cost("first","second"))
print('Path cost from \'first\' -> \'second\' -> \'third\' -> \'first\'')
print(g.path_cost(['first','second','third','first']))

Edge cost of 'first' -> 'second'
3
Path cost from 'first' -> 'second' -> 'third' -> 'first'


NameError: name 'g' is not defined

Our final and most interest method is the depth first traversal. This will result in a new graph of the given type (in our example, an Adjacency List) that is a spanning tree starting at a different vertex. Lets start with the depth first traversal around the vertex 'third' for both our undirected and directed graphs.

In [ ]:
print('***Undirected Graph***')
a.dft('third').to_gv()
print('\n')
print('***Directed Graph***')
b.dft('third').to_gv()

That is the end of it! Thank you for using my notebook, I hope you find it useful.

If you have any questions please email me at nicholas.soucy@maine.edu

Happy Coding!